In [1]:
import torch
import torch.nn as nn
from transformers import BertTokenizerFast
from src.utils.decodertokens import UNMTDecoderTokens
from src.models.Encoder import UNMTEncoder
from src.models.Decoder import LSTM_ATTN_Decoder
from torch.utils.data import DataLoader
from src.utils.dataloader import UNMTDataset, data_collate


In [ ]:
bert_pretrained_type = 'bert-base-multilingual-cased'
tokenizer = BertTokenizerFast.from_pretrained(bert_pretrained_type)
train_dataset = UNMTDataset(tokenizer = tokenizer,lang = 'hi',)
train_loader = DataLoader(train_dataset, batch_size = 32, collate_fn = lambda x: data_collate(x, tokenizer))

mydevice = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

mymode = 'Train'
mydecoder_hidden_dim = 768

In [3]:
myencoder = UNMTEncoder(bert_pretrained_type = bert_pretrained_type)
myembedding_layer = myencoder.bert.embeddings.word_embeddings



In [4]:
# sample_batch = [train_dataset[i] for i in range(4)]
# print(sample_batch[3])
# print("\n\n\nprinted sample batch\n\n\n")
# collated_batch = data_collate(sample_batch, tokenizer)
# print(collated_batch[1].shape)

In [ ]:
batch = next(iter(train_loader))
input_ids, attention_mask, noisy_input_ids, noisy_attention_mask,orig_sentence, word_idx = batch
print(input_ids.shape,"\n", attention_mask.shape,"\n", noisy_input_ids.shape,"\n", noisy_attention_mask.shape,"\n", orig_sentence,"\n", word_idx)

In [ ]:
encoder_output = myencoder(noisy_input_ids,attention_mask = noisy_attention_mask)
print(encoder_output.shape)

In [ ]:
mydecoder_hi = LSTM_ATTN_Decoder(lang = 'hi', tokenizer = tokenizer, 
                                     embedding_layer = myembedding_layer, mode=mymode,
                                     hidden_dim = mydecoder_hidden_dim)

In [10]:
decoder_output = mydecoder_hi(encoder_output,target_seq = input_ids, g_truth = False)

In [ ]:
print(decoder_output.shape)